In [1]:
# Now you can import the Deck class
import sys
sys.path.append('../src')

from game import Game
from player import Player


In [2]:
players = [Player('Player 1'), Player('Player 2'), Player('Player 3')]
game = Game(players)

In [3]:
game.start_game()

Player 1 drew card FIVE♦
Player 2 drew card TEN♣
Player 3 drew card EIGHT♣
Player 1 drew card FIVE♣
Player 2 drew card JOKER
Player 3 drew card JOKER
Player 1 drew card ONE♠
Player 2 drew card JOKER
Player 3 drew card TEN♠
Player 1 drew card SEVEN♠
Player 2 drew card TWO♠
Player 3 drew card FOUR♦
Player 1 drew card NINE♥
Player 2 drew card FOUR♣
Player 3 drew card JOKER
Player 2 played cards ['TEN♣']
Player 2 drew card SIX♣
Player 3 played cards ['EIGHT♣']
Player 3 drew card SEVEN♦
Player 1 played cards ['FIVE♦']
Player 1 drew card SEVEN♣
Turn 1
Player 2 played cards ['JOKER']
Player 2 drew card NINE♣
Player 3 played cards ['JOKER']
Player 3 drew card TWO♣
Player 1 played cards ['FIVE♣']
Player 1 drew card THREE♦
Turn 2
Player 2 played cards ['JOKER']
Player 2 drew card ONE♣
Player 3 played cards ['TEN♠']
Player 3 drew card JOKER
Player 1 played cards ['ONE♠']
Player 1 drew card JOKER
Turn 3
Player 2 played cards ['TWO♠']
Player 2 drew card EIGHT♥
Player 3 played cards ['FOUR♦']
Player

KeyboardInterrupt: Interrupted by user

In [4]:
game.scoreboard.round_log

[{<player.Player at 0x206b1a1b340>: (5, <Result.GOT_DESTROYED: 1>),
  <player.Player at 0x206b1a1bc40>: (19, <Result.NORMAL: 3>),
  <player.Player at 0x206b1a1bbb0>: (10, <Result.NORMAL: 3>)}]

In [7]:
game.players[0].calculate_score()

0

In [10]:
game.deal_cards()

Player 1 drew card SIX♦
Player 2 drew card SIX♣
Player 3 drew card SEVEN♦
Player 1 drew card SIX♥
Player 2 drew card TEN♠
Player 3 drew card JOKER
Player 1 drew card NINE♥
Player 2 drew card FIVE♣
Player 3 drew card FOUR♣
Player 1 drew card EIGHT♥
Player 2 drew card TWO♦
Player 3 drew card TEN♣
Player 1 drew card FOUR♠
Player 2 drew card JOKER
Player 3 drew card TWO♣


In [13]:
players[0].called_dhumbal = True

In [16]:
game.scoreboard.record_round(game)

In [17]:
game.scoreboard.scores

{<player.Player at 0x206b1a1b340>: 60,
 <player.Player at 0x206b1a1bc40>: 65,
 <player.Player at 0x206b1a1bbb0>: 56}

In [22]:
game.prepare_next_round(
    
)

Starting Round 2
Current Scores: {<player.Player object at 0x00000206B1A1B340>: 60, <player.Player object at 0x00000206B1A1BC40>: 65, <player.Player object at 0x00000206B1A1BBB0>: 56}


KeyboardInterrupt: Interrupted by user